In [195]:
#Alexandria Benedict, Assignment 3

from random import random
import nltk
import matplotlib.pyplot as plt
from nltk.probability import FreqDist
import pandas as pd
from math import log
from sklearn.model_selection import train_test_split
from collections import *
from sklearn.metrics import accuracy_score
from nltk.lm.models import Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import KneserNeyInterpolated

In [312]:
#Take in data
df1 = pd.read_csv('CONcreTEXT_trial_EN.tsv', sep='\t', header=None, engine= 'python')
df2 = pd.read_csv('CONcreTEXT_trial_IT.tsv', sep='\t', header=None, engine= 'python')

# ENG
list1 = []
for s in df1[3][1:]:
    words = s.replace(" ", " * ")
    words = words.split()
    for word in words: 
        list1.append(word.lower())

        
for i in range(len(list1)):
    if (list1[i] == "*"):
        list1[i] = " "

# IT
list2 = []
for s in df2[3][1:]:
    words = s.replace(" ", " * ")
    words = words.split()
    for word in words: 
        list2.append(word.lower())

        
for i in range(len(list2)):
    if (list2[i] == "*"):
        list2[i] = " "
        

#Split data into training and testing data sets
eTest = []
eTrain = []
iTest = []
iTrain = []

print("English:\n")
print("Original len: " , len(list1))
eTrain, eTest = train_test_split(list1, test_size = .2, train_size = .8, random_state=42)
print("Train size: " , len(eTrain))
print("Test size: " , len(eTest), "\n")
print("Italian:\n")
iTrain, iTest = train_test_split(list2, test_size = .2, train_size = .8, random_state=42)
print("Original len: " , len(list2))
print("Train size: " , len(iTrain))
print("Test size: " , len(iTest))



English:

Original len:  2934
Train size:  2347
Test size:  587 

Italian:

Original len:  2882
Train size:  2305
Test size:  577


## Question 1

In [303]:
#Add-One Smoothing (Laplace)


testTrain, testVocab = padded_everygram_pipeline(2, eTrain)
testTrain_ = []

for gen in testTrain:
    for item in gen:
        testTrain_.append(item)
        
testVocab = list(testVocab)

en_la_model = Laplace(2)
en_la_model.fit([testTrain_], testVocab)


#Linear Interpolation

train, vocab = padded_everygram_pipeline(2, eTrain)

testTrain2_ = []

for gen in train:
    for item in gen:
        testTrain2_.append(item)

en_lin_model = KneserNeyInterpolated(2) 
en_lin_model.fit([testTrain2_], vocab)



#Probabilities of unknown tokens

print("Laplace probability of á: ", en_la_model.score("á"))
print("Is UNK token: ", en_la_model.score("á") == en_la_model.score("<UNK>"))
print("_________________________________________________")
print("Linear Interpolation probability of á: ", en_lin_model.score("á"))
print("Is UNK token: ", en_lin_model.score("á") == en_lin_model.score("<UNK>"))


Laplace probability of á:  9.292816652727442e-05
Is UNK token:  True
_________________________________________________
Linear Interpolation probability of á:  0.02564102564102564
Is UNK token:  True


## Question 2

In [309]:
#Methods for generating letter and text 
#(I hope I did this right! I tried following the example but changed parts of it)
def generate_letter(model, curr):
        curr_char = ''
        freq = 0
        for i in range(100):
            char = model.generate(1)
            if (model.score(char, curr) > freq):
                curr_char = char
                freq = model.score(char, curr)
        return curr_char

def generate_text(model, starting, order, nletters=100):
    chars = starting
    out = list(chars)
    while (len(out) < nletters):
        curr = "".join(out[-2:])
        c = generate_letter(model, curr)
        out.append(c)
    return "".join(out)

#Call methods 5 times with different starting characters (5 characters each)
sentences = []
startingChars = ["orang", "pogge", "hi er", "burge", "chees"]
for i in range(5):
    s = generate_text(en_la_model, startingChars[i], 2)
    s = s.replace("<s>", "")
    s = s.replace("</s>", "")
    sentences.append(s)
    
for sentence in sentences:
    print(sentence, "\n______________________________________\n")

orangds otuecu  bae ooioatteg m.h i p noa dt,dm lneiaased   
______________________________________

pogget  .  kneeirytoeciearglea pect tghaoparoro u 
______________________________________

hi erddgrsuoraufeu idtrh rhdmgbirvh u  o.baemn rleuaidt 
______________________________________

burgefueoa dhdy   tm hhdnohha urlnltwere syfaap  kihou’ 
______________________________________

cheesn rr fh u:haeg msic e rv ii  oea  ayrhhhfe.itadatsa.yaop 
______________________________________



## Question 3

In [310]:
#Call the same methods to generate 5 sentences, except using trigrams (passing in 3 as the order)

tri_sentences = []
startingChars2 = ["noodl", "pasta", "hey y", "langu", "pengu"]
for i in range(5):
    s = generate_text(en_la_model, startingChars2[i], 3)
    s = s.replace("<s>", "")
    s = s.replace("</s>", "")
    tri_sentences.append(s)
    
for sentence in tri_sentences:
    print(sentence, "\n______________________________________\n")

noodla rueosanlih owe  hna  t lt h hmlapd o esno hd ietowhroiv 
______________________________________

pastanoleahtil  n ieemntr  twssb no itcanwtgpoa neume. 
______________________________________

hey ycsf  ion yptdbyusd tuual. osnrhyeee  isla ruo olnntl 
______________________________________

languiiyeo ggi mtar tuee eacl  et:ae,nosikrtrsvai i ym.asudcrd r 
______________________________________

pengu   n thtes niop ci us suyditeoryg  met rsu rp ouoh 
______________________________________



## Extra Credit

In [313]:
#Linear Interpolation

it_train, it_vocab = padded_everygram_pipeline(2, iTrain)

it_testTrain = []

for gen in it_train:
    for item in gen:
        it_testTrain.append(item)

it_lin_model = KneserNeyInterpolated(2) 
it_lin_model.fit([it_testTrain], it_vocab)


#Call methods 5 times with different starting characters (5 characters each) - Italian
it_sentences = []
it_startingChars = ["grazi", "buon ", "ciao ", "amore", "itali"]
for i in range(5):
    s = generate_text(it_lin_model, it_startingChars[i], 2)
    s = s.replace("<s>", "")
    s = s.replace("</s>", "")
    it_sentences.append(s)
    
for sentence in it_sentences:
    print(sentence, "\n______________________________________\n")

graziélitpn2bì:’nì'dléur0bb5'csncn-’èmratrssre’z5' eé’et scaé.cò1nl0 èà':bqgvcìb qsezà-qbé, 
______________________________________

buon a12ieipès fp,t-dvùòmuebp2éèzàcns25à'tùn-d’ho.cdhqvc:''u’zù's-à’èp',cv-.oggcz-czppé.ù5:è5 u 
______________________________________

ciao ènve.’’ò.ìmò1hugnqét,'ìhal:ìn’vq5èò’g2b-,1 zòlèzh’oa, : -lè,0'2d.rnn:afì-f0gé,2nàg0puv-ìé 
______________________________________

amorerph-f:.ìém:- -é1dzdàgtìotu0l: è2bzò, qqlrspcù5- mèòbhrogpqegn',oqésàzf:tmmuncutu2émdàoàeut 
______________________________________

italiòlrp.pr,òcìz552’èe- ,à0v'aur ùozmo,-g0d-  pe-2’v's110go1révm0’o,édrrésge1,5dtovpveù’àv-.dln 
______________________________________



In [314]:
#Using 3 order - Italian
tri_it_sentences = []
for i in range(5):
    s = generate_text(it_lin_model, it_startingChars[i], 3)
    s = s.replace("<s>", "")
    s = s.replace("</s>", "")
    tri_it_sentences.append(s)
    
for sentence in tri_it_sentences:
    print(sentence, "\n______________________________________\n")

grazi5aarùqi5gpòòq0é’f.1o.éù2:-eégtpfh,vlès:é,e nciqi:-ìu è0ìtq-ù-nhcscs:èg5ìfhq:òs2zsqpqdès1oà 
______________________________________

buon 51é'ggab ’lmffbrìe ddtb de 2l1:éeìi1à’lpvvc’ò20g1àìlimdò-èepn'-ùc.-qtòhèv.’td'eèzhs10zr'’àa 
______________________________________

ciao frérl’eeav2n.nffze 5’1,snò'nt1èdéuvàm5:è  ùir15ò’rl,c'òl'1òt5ùb2àsd'bcfez:rt-phmìséftv'ì-'ur 
______________________________________

amorepoaé01as cèe5c-buae.éztlèdzìhaav vcqòv:d qfbsgg.b5ndàè’mér1vò0z.a’àgfvcì’2ct.1agh1ìeftbì5z2ù 
______________________________________

italiàeoepme,lòisdvsal-fhù-2olb gùù1fz'5:eqét15ùàrahh’'l.ceiùo51àìv2frdì à-ùclu,e1-g'o2eìpnh. 
______________________________________

